In [ ]:
import shelve

dict1 = {"a": 1, "b": 2, "c": 3}
dict2 = {1: "a", 2: "b", 3: "c"}

log_file = 'dictionary-shelf'

with shelve.open(log_file, "c") as shelf:
    for i in range(3):
        shelf_name = 'dict_'+str(i)
        shelf[shelf_name] = dict1

with shelve.open(log_file, 'w') as shelf:
    for i in range(2): 
        shelf_name = 'second_dict_'+str(i)
        shelf[shelf_name] = dict2

    del shelf['dict_1'] # how to delete!!!!!!  

# how to read: 

'''
with shelve.open(log_file, "r") as shelf:
    dict3 = shelf["first"]
    dict4 = shelf["second"]
'''

with shelve.open(log_file, 'r') as shelf: 
    print(list(shelf.keys()))

['second_dict_0', 'dict_0', 'dict_2', 'second_dict_1']


# Notes on shelve

## Shelve Opening Flag Meanings

| Value      | Meaning |
| ----------- | ------------------- |
| `'r'` | Open existing database for reading only (default)|
| `'w'` | Open existing database for reading and writing |
| `'c'` | Open database for reading and writing, creating it if it doesn’t exist |
| `'n'`| Always create a new, empty database, open for reading and writing |

## Operation Notes
* Open new shelve file: 
```python 
with shelve.open(file_name, "c") as shelf:
        shelf['shelf_name'] = example_dictionary
```

* Open with read write capabilities shelve file that already exists, and delete a dictionary within it: 
```python
with shelve.open(file_name, 'w') as shelf:
    if 'example_key' in shelf: 
        del shelf['example_key']
```

* Open shelve file, print key, and read dictionaries within it: 
```python
with shelve.open(file_name, 'r') as shelf:
    print(list(shelf.keys()))

    example_dictionary = shelf['example_name']
```



## Other Notes
* A “shelf” is a persistent, dictionary-like object
* The difference with “dbm” databases is that the values (not the keys!) in a shelf can be essentially arbitrary Python objects — anything that the pickle module can handle.
* The shelve allows you to store pickled objects ... in a perseistent manner. It is useful to store large amount of data because Python performs *shelf lookups as needed* instead of loading the entire shelf in memory.


In [26]:

def download(
    ticstr: str, 
    outdir: str = 'none', 
    logfile: str = 'none'): 
    ''' 
    Args:
        outdir: directory where lightcurves will be saved. If not set, data will not be saved.  --> FIX!
        ticstr: e.g., 'TIC 441420236'.  
    Returns: 
        raw_list: list of light curve ojects that mean criteria but have not been processed (i.e. not detrended, normalized, or sigma-clipped) 
    '''
    import numpy as np 
    import lightkurve as lk 

    # get the light curve
    lcc = lk.search_lightcurve(ticstr).download_all()

    # select only the two-minute cadence SPOC-reduced data; convert to a list.
    # note that this conversion approach works for any LightCurveCollection
    # returned by lightkurve -- no need to hand-pick the right ones.  the exact
    # condition below says "if the interval is between 119 and 121 seconds,
    # take it".
    raw_list = [_l for _l in lcc
            if
            _l.meta['ORIGIN']=='NASA/Ames'
            and
            np.isclose(
                120,
                np.nanmedian(np.diff(_l.remove_outliers().time.value))*24*60*60,
                atol=1
            )
    ]

    raw_list = [_l for _l in raw_list if _l.meta['FLUX_ORIGIN']=='pdcsap_flux']

       
    if logfile != 'none': 
        import shelve
        with shelve.open(log_file, "c") as shelf:
            log_dict = {'sectors', len(raw_list)}
            dict_name = 'download:'+ticstr.replace(' ','')
            shelf[dict_name] = log_dict
        
    return raw_list


In [37]:
raw_list = download('TIC 165818534', logfile='test-shelf')

import shelve
with shelve.open('test-shelf', "c") as shelf:
    log_dict = {'sectors', len([3,3,3,3])}
    dict_name = 'TIC 165818534'.replace(' ','_')
    shelf[dict_name] = log_dict

saved?


In [38]:
with shelve.open('test-shelf', 'r') as shelf:
    print(shelf['TIC_165818534']) 

{'sectors', 4}
